In [1]:
import pandas as pd
from web3 import Web3

In [2]:
# Set up your Ethereum provider
rpc_url = 'https://rpc.ankr.com/arbitrum'
web3_provider = Web3(Web3.HTTPProvider(rpc_url))

In [3]:
connected = web3_provider.is_connected()

In [4]:
# Check if the connection is successful
if connected == True:
    print("Connected")

Connected


In [5]:
# ABI of the Chainlink price feed contract
abi = '''
[
    {
        "constant": true,
        "inputs": [],
        "name": "latestRoundData",
        "outputs": [
            {"name": "roundId", "type": "uint80"},
            {"name": "answer", "type": "int256"},
            {"name": "startedAt", "type": "uint256"},
            {"name": "updatedAt", "type": "uint256"},
            {"name": "answeredInRound", "type": "uint80"}
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": true,
        "inputs": [
            {"name": "_roundId", "type": "uint80"}
        ],
        "name": "getRoundData",
        "outputs": [
            {"name": "roundId", "type": "uint80"},
            {"name": "answer", "type": "int256"},
            {"name": "startedAt", "type": "uint256"},
            {"name": "updatedAt", "type": "uint256"},
            {"name": "answeredInRound", "type": "uint80"}
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    }
]
'''


In [6]:
# Chainlink ETH/USD price feed contract address
contract_address = '0x639Fe6ab55C921f74e7fac1ee960C0B6293ba612'

# Instantiate the contract
contract = web3_provider.eth.contract(address=contract_address, abi=abi)

In [7]:
# Get the latest round data to determine the latest round ID
latest_data = contract.functions.latestRoundData().call()
latest_round_id = latest_data[0]
latest_round_id


18446744073710177366

In [8]:
latest_data

[18446744073710177366,
 314690000000,
 1720909612,
 1720909612,
 18446744073710177366]

In [ ]:
# List to store historical data
historical_data = []

# Fetch historical data for each round
for round_id in range(latest_round_id, 1, -1):
    try:
        round_data = contract.functions.getRoundData(round_id).call()
        round_info = {
            'roundId': round_data[0],
            'price': round_data[1],
            'startedAt': round_data[2],
            'updatedAt': round_data[3],
            'answeredInRound': round_data[4]
        }
        historical_data.append(round_info)
    except Exception as e:
        print(f"Could not fetch data for round {round_id}: {e}")
        if e == 'no data':
            break

# Convert to DataFrame
df = pd.DataFrame(historical_data)

# Save to CSV
df.to_csv('chainlink_historical_data.csv', index=False)

print("Historical data saved to chainlink_historical_data.csv")